<a href="https://colab.research.google.com/github/MamiyaA/Biomechanics-and-function-of-proprioceptors-Python-analysis-functions/blob/master/make_video_with_filtered_tdTomato_signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python function for creating a video that shows the movements of cell during imaging.

## Uses OpenCV to create a video of fluorescent images overlayed on top of the thresholded image.

## Load the output from the "apply_3D_gaussian_to_ScanImageFile" (filtered tdTomato and GCaMP images) and make a video of tdTomato signal.

- #### Specify the threshold for the mask overlay.
- #### Runs as a google colab notebook.
- #### Data is located on google drive.

### Connect to a hosted runtime then import necessary packages

In [ ]:
#import the necessary packages.
import numpy as np
import os
import fnmatch
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import pickle
from skimage.segmentation import watershed
from skimage.measure import regionprops_table
import cv2

### Mount the Google Drive (Shared drives)

In [ ]:
#link to the google drive. Need to follow the authorization instructions.
from google.colab import drive
drive.mount('/content/drive')

### Use the output from a Function: apply_3D_gaussian_to_ScanImageFile

- #### The function is defined in a google colab notebook located in the same directory as this notebook. 

- #### It saves a pickle file that we are going to use here. It contains signals from GCaMP and tdTomato that are filtered with 3D gaussian filter.

- #### signals from different depth are averaged for this matrix
    

### Define a function that loads the filtered data and make the overlay video

In [ ]:
#a function to load the filtered data and make an .avi movie
#overlay a mask defined by mask_threshold.
#frames_per_second defines how many fps for the video.
#
#min_range and max_range defines the min and max for the tdTomato image
#0 to 3500 seems to work well.

def load_filtered_data_make_overlay_video(input_file,mask_threshold,frames_per_second,min_range,max_range):
    #Load the file
    infile=open(input_file,'rb')
    tdTomato_Filtered=pickle.load(infile)
    infile.close()
    
    
    
    #Make a video with the original tdTomato signal plus overlay of the thresholded image
    video_name = (input_file+"tdTomato_signal_overlay.avi")
   
    video = cv2.VideoWriter(video_name,cv2.VideoWriter_fourcc(*'mp4v'),frames_per_second,(tdTomato_Filtered.shape[2],tdTomato_Filtered.shape[1]))

    #Number of frames in the data
    total_frames = tdTomato_Filtered.shape[0]

    #For making video, ignore the negative numbers.
    tdTomato_Filtered[tdTomato_Filtered<0]=0
    range_adjusted_tdTomato=((tdTomato_Filtered-min_range)/max_range)*255
    
    #For overlay. Hard coded for now.
    alpha=0.1
    beta=(1.0-alpha)
    thresholded_image=tdTomato_Filtered>=mask_threshold

    for video_frame in range(total_frames):
        frame = cv2.applyColorMap(np.uint8(range_adjusted_tdTomato[video_frame,:,:]),cv2.COLORMAP_PARULA)
    
        frame_threshold = cv2.applyColorMap(np.uint8(thresholded_image[video_frame,:,:]*250),cv2.COLORMAP_WINTER)

        add_image = cv2.applyColorMap(np.uint8(range_adjusted_tdTomato[video_frame,:,:]),cv2.COLORMAP_PARULA)

        cv2.addWeighted(frame_threshold,alpha,frame,beta,0.0,add_image)
        video.write(add_image)

    video.release()

### Get the path to the filtered data

In [ ]:
#Get the file path to the filtered data

filtered_file_path=[]
#Path for the image files
path = '/content/drive/Shared drives/Data'

#Go through all the files in a directory (probably includes subidirectoies as well?)
for file_name in os.listdir(path):
    if fnmatch.fnmatch(file_name,'*tdTomato_Filtered'):
        filtered_file_path.append(os.path.join(path,file_name))
        
filtered_file_path_sorted=sorted(filtered_file_path)

#Print to make sure
for f in filtered_file_path_sorted:
    print(f)
    


### Run for all the filtered files in the Data folder

In [ ]:
#Print files names to track the progress
for filtered_files in filtered_file_path_sorted:
  print(filtered_files)
  load_filtered_data_make_overlay_video(filtered_files,90,30,0,1000)

